In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json, os, time, re
from time import sleep
from pprint import pprint
from urllib import parse
from selenium.webdriver.common.keys import Keys
# 隨機取得 User-Agent
from fake_useragent import UserAgent
import glob
from concurrent.futures import ThreadPoolExecutor
ua = UserAgent()

searchlist = []

'''手語辭典的網址'''
url = 'https://twtsl.ccu.edu.tw/TSL/index.php'

# 沒有放置 txt 檔的資料夾，就建立起來
folderPath = 'sign language'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [11]:
def setup_driver_1():
      my_options = webdriver.ChromeOptions()
      my_options.add_argument('--start-maximized')
      my_options.add_argument('--incognito')
      my_options.add_argument('--disable-popup-blocking')
      my_options.add_argument(f'--user-agent={ua.random}')
      return webdriver.Chrome(
      options = my_options,
      service = Service(ChromeDriverManager().install())
      )
def setup_driver_2():
      my_options = webdriver.ChromeOptions()
      my_options.add_argument('--start-maximized')
      my_options.add_argument('--incognito')
      my_options.add_argument('--disable-popup-blocking')
      my_options.add_argument(f'--user-agent={ua.random}')
      my_options.add_argument("--headless")
      my_options.add_argument("--no-sandbox")
      my_options.add_argument("--disable-dev-shm-usage")
      return webdriver.Chrome(
      options = my_options,
      service = Service(ChromeDriverManager().install())
      )
def getsearchlist_1():
    driver = setup_driver_1()
    driver.get(url)
    s=0
    driver.find_element(By.CSS_SELECTOR, 'button#pinOpener').click()
    sleep(1)
    for a in range(len(driver.find_elements(By.CSS_SELECTOR, 'div.col-md-3 > a'))):
        sleep(1)
        writenumber = driver.find_elements(By.CSS_SELECTOR, 'div.col-md-3 > a')[s]
        writenumber.click()
        sleep(1)
        while True:
               try:
                  # 等待篩選元素出現
                  WebDriverWait(driver, 10).until(
                  EC.presence_of_element_located( 
                  (By.CSS_SELECTOR, 'div#queryDiv> a[href]') 
                       )
                         )
                  elms = driver.find_elements(By.CSS_SELECTOR, 'div#queryDiv> a[href]') 
                  for elm in elms:
                    sleep(0.1)
                    searchlist.append(elm.text)
               except TimeoutException:
                    print("等待逾時 手語")
                    break     
               try:
                  unclickable = driver.find_elements(By.CSS_SELECTOR, 'li.page-item')[2]
                  if 'disabled' in  unclickable.get_attribute('class'):
                      break
                  else:
                      next_button = WebDriverWait(driver, 10).until(
                      EC.element_to_be_clickable((By.LINK_TEXT, "下一頁"))
                       )
                      driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                      sleep(0.5)
                      driver.execute_script("arguments[0].click();", next_button)
                    
               except TimeoutException:
                  print("等待逾時 下一頁")
                  break
        writenumber_search = driver.find_element(By.CSS_SELECTOR, 'button#pinOpener')
        writenumber_search.click()
        s += 1
    driver.quit()
    print(len(searchlist))
    with open(f"{folderPath}/searchlist.json", "w", encoding="utf-8") as f:
            json.dump(searchlist, f, ensure_ascii=False, indent=2)    
def getsearchlist_2():
    driver.get(url)
    n=0
    x=0
    driver.find_element(By.CSS_SELECTOR, 'button#handOpener').click()
    sleep(2)
    for a in range(len(driver.find_elements(By.CSS_SELECTOR, 'div.btn-group > button'))):
        sleep(2)
        fingernumber = driver.find_elements(By.CSS_SELECTOR, 'div.btn-group > button')[n]
        fingernumber.click()
        x = 0
        for b in range(len(driver.find_elements(By.CSS_SELECTOR, 'div#handJPGContent  img.m-1'))):
            sleep(2)
            fingerpicture = driver.find_elements(By.CSS_SELECTOR, 'div#handJPGContent  img.m-1')[x]
            fingerpicture.click()
            sleep(1)
            allword = driver.find_element(By.CSS_SELECTOR, 'div#selectHandJPG a')
            allword.click()
            sleep(1)
            cancel = driver.find_elements(By.CSS_SELECTOR, "div.modal-footer button")[1]
            cancel.click()
            while True:
               try:
                  # 等待篩選元素出現
                  WebDriverWait(driver, 10).until(
                  EC.presence_of_element_located( 
                  (By.CSS_SELECTOR, 'div#handJPGContentWord > a[href]') 
                       )
                         )
                  elms = driver.find_elements(By.CSS_SELECTOR, 'div#handJPGContentWord > a[href]') 
                  for elm in elms:
                    sleep(0.1)
                    searchlist.append(elm.text)
               except TimeoutException:
                    print("等待逾時 手語")
                    continue     
               try:
                  # 等待篩選元素出現
                  sleep(2)
                  unclickable = driver.find_elements(By.CSS_SELECTOR, 'li.page-item')[2]
                  if 'disabled' in  unclickable.get_attribute('class'):
                      break
                  else:
                      next_button = WebDriverWait(driver, 10).until(
                      EC.element_to_be_clickable((By.LINK_TEXT, "下一頁"))
                       )
                      driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                      sleep(0.5)
                      driver.execute_script("arguments[0].click();", next_button)
                    
               except TimeoutException:
                  print("等待逾時 下一頁")
                  continue
            searchhand = driver.find_element(By.CSS_SELECTOR, 'button#handOpener')
            searchhand.click()
            sleep(2)
            fingernumber = driver.find_elements(By.CSS_SELECTOR, 'div.btn-group > button')[n]
            fingernumber.click()
            sleep(2)
            x += 1
        n += 1
    print(len(searchlist))
    
def getmainlinks(searchlist, thread_id):
    try:
        listData_word= []
        listData_sentence= []
        driver = setup_driver_2()
        driver.get(url)
        sleep(3)
        for word in searchlist:
            g = 0
            z = 0
            driver.execute_script("document.getElementById('searchBar').value = ''")
            while True:
                try:
                   WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located( 
                   (By.CSS_SELECTOR, 'input#searchBar') 
                               )
                                 )
                   txtInput = driver.find_element(By.CSS_SELECTOR, "input#searchBar")
                   break 
                except TimeoutException:
                    print("等待逾時 搜尋")
                    pass
            txtInput.send_keys(f"{word}")
            sleep(1)

            while True:
                try:
                   WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located( 
                   (By.CSS_SELECTOR, 'button#send') 
                               )
                                     )
                   txtInput = driver.find_element(By.CSS_SELECTOR, "button#send")
                   txtInput.click()
                    
                except TimeoutException:
                   print("等待逾時 搜尋紐")
                   break
            sleep(3)

            while True:
                try:
                   WebDriverWait(driver, 10).until(
                   EC.presence_of_element_located( 
                   (By.CSS_SELECTOR, 'video#wordMovie > source') 
                               )
                                 )
                   link_word = driver.find_element(By.CSS_SELECTOR, 'video#wordMovie > source').get_attribute('src')
                   break 
                except TimeoutException:
                   print("等待逾時 單字影片")
                   pass
                
            try:    
               WebDriverWait(driver, 0.1).until(
               EC.presence_of_element_located( 
               (By.CSS_SELECTOR, 'div.sentenceHead') 
                           )
                             )
               word_sentence = driver.find_elements(By.CSS_SELECTOR, 'div.sentenceHead')
               WebDriverWait(driver, 0.1).until(
               EC.presence_of_element_located( 
               (By.CSS_SELECTOR, 'div#sentenceVideo  video source') 
                           )
                             )
               link_sentence = driver.find_elements(By.CSS_SELECTOR, 'div#sentenceVideo  video source') 
               for b in range(len(word_sentence)):
                  listData_sentence.append({
                    "sentence" : word_sentence[z].text,
                    "link" :  parse.unquote( link_sentence[z].get_attribute('src'))})
                  z += 1  
            except TimeoutException:
                pass
            listData_word.append({
                "word" : word,
                "link" : parse.unquote(link_word) 
                })    
            sleep(2)
        driver.quit()
        with open(f"{folderPath}/thread_{thread_id}_word.json", "w", encoding="utf-8") as f:
            json.dump(listData_word, f, ensure_ascii=False, indent=2)
        with open(f"{folderPath}/thread_{thread_id}_sentence.json", "w", encoding="utf-8") as f:
            json.dump(listData_sentence, f, ensure_ascii=False, indent=2)    
        print(f"[{thread_id}] Done, saved {len(listData_word)},{len(listData_sentence)} results.")
    except Exception as e:
        print(f"[Thread-{thread_id}] 發生錯誤：{e}")
def worktogether():    
  def chunkify(lst, n):
      return [lst[i::n] for i in range(n)]
  num_threads = 3
  word_groups = chunkify(searchlist, num_threads)
  with open(f"{folderPath}/searchlist.json", "r", encoding="utf-8") as f:
        searchlist = json.load(f)
  # 多線程跑起來
  with ThreadPoolExecutor(max_workers=num_threads) as executor:
      for i, group in enumerate(word_groups):
          executor.submit(getmainlinks, group, i)
def savejason():
    # 找出所有結果檔案
  file_list_word = glob.glob(f"{folderPath}/thread_*_word.json")
  print(file_list_word)
  file_list_sentence = glob.glob(f"{folderPath}/thread_*_sentence.json")
  print(file_list_sentence)
  all_data_word = []
  all_data_sentence = []
# 讀取並合併所有檔案
  for file in file_list_word:
      with open(file, "r", encoding="utf-8") as f:
        data_word = json.load(f)
        all_data_word.extend(data_word)  # 加到總資料中
  for file in file_list_sentence:
      with open(file, "r", encoding="utf-8") as f:
        data_sentence = json.load(f)
        all_data_sentence.extend(data_sentence)  # 加到總資料中
  # 儲存合併結果
  with open(f"{folderPath}/all_results_word.json", "w", encoding="utf-8") as f:
    json.dump(all_data_word, f, ensure_ascii=False, indent=2)
  with open(f"{folderPath}/all_results_sentence.json", "w", encoding="utf-8") as f:
    json.dump(all_data_sentence, f, ensure_ascii=False, indent=2)
  print(f"成功合併 {len(file_list_word)},{len(file_list_sentence)} 個檔案，總共 {len(all_data_word)},{len(all_data_sentence)} 筆資料！")    
                                                     
       
    
    
   

    

In [12]:
if __name__ == "__main__":
    time1 = time.time()
    getsearchlist_1()
    worktogether()
    savejason()
    print(f"執行總花費時間: {time.time() - time1}")
    
    
    

[0] Done, saved 1367,280 results.
[2] Done, saved 1367,266 results.
[1] Done, saved 1367,291 results.
['sign language\\thread_0_word.json', 'sign language\\thread_1_word.json', 'sign language\\thread_2_word.json', 'sign language\\thread_3_word.json', 'sign language\\thread_4_word.json']
['sign language\\thread_0_sentence.json', 'sign language\\thread_1_sentence.json', 'sign language\\thread_2_sentence.json', 'sign language\\thread_3_sentence.json', 'sign language\\thread_4_sentence.json']
成功合併 5,5 個檔案，總共 4103,837 筆資料！
執行總花費時間: 10207.74883389473


In [10]:
#searchlist = ["夜晚","一","一些","一筆勾銷","一致","一人一半","伴","完成"]
    #with open(f"{folderPath}/searchlist.json", "w", encoding="utf-8") as f:
            #json.dump(searchlist, f, ensure_ascii=False, indent=2)

In [ ]:
#函式一
with open(f"{folderPath}/searchlist.json", "w", encoding="utf-8") as f:
            json.dump(searchlist, f, ensure_ascii=False, indent=2)#儲存資料
#函式二   
with open(f"{folderPath}/searchlist.json", "r", encoding="utf-8") as f:
        searchlist = json.load(f)#打開資料，後面寫主程式，